In [4]:
import os
import glob

In [5]:
import random
import cv2

In [6]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import models


In [18]:
from pathlib import Path

In [7]:
data_path = 'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\'

In [8]:
import numpy as np
from scipy import ndimage
import scipy.signal as sig
import nibabel as nib
import matplotlib.pyplot as plt
import os
from PIL import Image
import random
import time



def bbox(gt_1, gt_2, extra=10):
    
    # We have 2 segmentation instances for each patient. We want to use a single bbox for all the images.
    # The segmentation pixels are either not 0 (where the heart is) or 0, we use this to get all the pixels where the heart should be.
    # From these pixels we can get the x,y boundaries of the box in which these pixels are
    # From these 4 numbers (y_min,y_max,x_min,x_max) we can construct the bbox
    
    if gt_1.shape != gt_2.shape:
        print('Ground truth image shape missmatch')
        
    max_list=[0,256,0,256]
    
    for j in range(gt_1.shape[2]):
        for y in range(gt_1.shape[0]):
            for x in range(gt_1.shape[1]):

                if gt_1[y][x][j] != 0 or gt_2[y][x][j] != 0:
                    if y >= max_list[0]:
                        max_list[0] = y
                    if y <= max_list[1]:
                        max_list[1] = y
                    if x >= max_list[2]:
                        max_list[2] = x
                    if x<= max_list[3]:
                        max_list[3] = x
                    
    max_list[0] = max_list[0] + extra
    max_list[1] = max_list[1] - extra
    max_list[2] = max_list[2] + extra
    max_list[3] = max_list[3] - extra
                    
    return max_list

def crop_img(img, bbox_boundaries, kernel_size=None):
    
    # crops the image based on the bbox
    
    if kernel_size == None:
        y_max = bbox_boundaries[0]
        y_min = bbox_boundaries[1]
        x_max = bbox_boundaries[2]
        x_min = bbox_boundaries[3]
        
    else:
        y_max = int(bbox_boundaries[0] / kernel_size)
        y_min = int(bbox_boundaries[1] / kernel_size)
        x_max = int(bbox_boundaries[2] / kernel_size)
        x_min = int(bbox_boundaries[3] / kernel_size)
        
    crop_img = np.zeros((img.shape[0],img.shape[1]))
    
    for y in range(img.shape[0]):
        for x in range(img.shape[1]):
            if y_min < y and y_max > y:
                if x_min < x and x_max > x:
                    crop_img[y][x] = img[y][x]
                    
    crop_img = crop_img[y_min:y_max,x_min:x_max]            
    return crop_img

In [9]:
def read_from_folder(data_path:str,folder:str,num:int):
    files_list = []
    gt_list = []
    for i in range(1,num):
        if i < 10:
            patient = 'patient00' + str(i)
            
        elif 9 < i < 100:
            patient = 'patient0' + str(i)   
            
        else:
            patient = 'patient' + str(i)
            
        gt_file_path = glob.glob(os.path.join(data_path,folder,patient,'*_gt.nii.gz'))
        file_path = glob.glob(os.path.join(data_path,folder,patient,'*_4d.nii.gz'))
        files_list.append((file_path[0],gt_file_path,patient))
        
    return files_list

In [10]:
def patient_train_split(patient_files_list,train_split=0.7, val_split=0.2, test_split=0.1):
    
    #70/20/10 %
    #splits our 100 patients into train, validation and test sets
    
    L = 100
    train_split = int(train_split * L)
    val_split = int(val_split * L)
    test_split = int(test_split * L)
    
            
    random.shuffle(patient_files_list)

    train_list = patient_files_list[:train_split]
    val_list = patient_files_list[train_split:train_split + val_split]
    test_list = patient_files_list[train_split + val_split:]
    
    return (train_list, val_list, test_list)

In [19]:
def reshape(img):
    HEIGHT = 256
    WIDTH = 256
    SHAPE = (HEIGHT, WIDTH)

    image = cv2.resize(img, (WIDTH,HEIGHT), interpolation=cv2.INTER_CUBIC)
    return image

In [15]:
def get_2d_images_gt(saving_path:str,files_path_list:list):
    count = 0
    for file,gt_path,patient in files_path_list:
        nii_img  = nib.load(file)
        nii_data = nii_img.get_fdata()
        
        nii1_img  = nib.load(gt_path[0])
        gt1_img = nii1_img.get_fdata()
        
        nii12_img  = nib.load(gt_path[1])
        gt12_img = nii12_img.get_fdata()
        
        box = bbox(gt1_img,gt12_img)
        for z in np.ndindex(nii_data.shape[2]):
            for t in np.ndindex(nii_data.shape[3]):
                img_2d = nii_data[:,:,z,t]
                crop_img_2d = crop_img(img_2d,box)
                reshape_img = reshape(crop_img_2d)
                saving_name = patient + '_' + str(z[0]+1) + '_' + str(t[0]+1) + '.jpg'
                Path(saving_path).mkdir(parents=True, exist_ok=True)
                cv2.imwrite(os.path.join(saving_path,saving_name), reshape_img)
                
        count += 1        
        print(f'Done:{len(files_path_list)}/{count}')
    print(f'All done')

In [152]:
training_files_list = read_from_folder(data_path,'training',101)
training_files_list

[('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient001\\patient001_4d.nii.gz',
  'patient001'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient002\\patient002_4d.nii.gz',
  'patient002'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient003\\patient003_4d.nii.gz',
  'patient003'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient004\\patient004_4d.nii.gz',
  'patient004'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient005\\patient005_4d.nii.gz',
  'patient005'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient006\\patient006_4d.nii.gz',
  'patient006'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient007\\patient007_4d.nii.gz',
  'patient007'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient008\\patient008_4d.nii.gz',
  'patient008'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient009\\patient009_4d.nii.gz',
  'patien

In [13]:
train_files_list,val_files_list,test_files_list = patient_train_split(training_files_list)
train_files_list

[('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient088\\patient088_4d.nii.gz',
  ['C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient088\\patient088_frame01_gt.nii.gz',
   'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient088\\patient088_frame12_gt.nii.gz'],
  'patient088'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient020\\patient020_4d.nii.gz',
  ['C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient020\\patient020_frame01_gt.nii.gz',
   'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient020\\patient020_frame11_gt.nii.gz'],
  'patient020'),
 ('C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient084\\patient084_4d.nii.gz',
  ['C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient084\\patient084_frame01_gt.nii.gz',
   'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\training\\patient084\\patient084_frame10_gt.nii.gz'],
  'patient084'),
 ('C:\\Users\\homol\\Documents\\DL\\D

In [14]:
len(train_files_list),len(val_files_list),len(test_files_list)

(70, 20, 10)

In [16]:
saving_train_path = 'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\GT_TRAIN\\'

In [21]:
get_2d_images_gt(saving_train_path,train_files_list)

Done:70/1
Done:70/2
Done:70/3
Done:70/4
Done:70/5
Done:70/6
Done:70/7
Done:70/8
Done:70/9
Done:70/10
Done:70/11
Done:70/12
Done:70/13
Done:70/14
Done:70/15
Done:70/16
Done:70/17
Done:70/18
Done:70/19
Done:70/20
Done:70/21
Done:70/22
Done:70/23
Done:70/24
Done:70/25
Done:70/26
Done:70/27
Done:70/28
Done:70/29
Done:70/30
Done:70/31
Done:70/32
Done:70/33
Done:70/34
Done:70/35
Done:70/36
Done:70/37
Done:70/38
Done:70/39
Done:70/40
Done:70/41
Done:70/42
Done:70/43
Done:70/44
Done:70/45
Done:70/46
Done:70/47
Done:70/48
Done:70/49
Done:70/50
Done:70/51
Done:70/52
Done:70/53
Done:70/54
Done:70/55
Done:70/56
Done:70/57
Done:70/58
Done:70/59
Done:70/60
Done:70/61
Done:70/62
Done:70/63
Done:70/64
Done:70/65
Done:70/66
Done:70/67
Done:70/68
Done:70/69
Done:70/70
All done


In [23]:
saving_val_path = 'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\GT_VAL\\'
get_2d_images_gt(saving_val_path,val_files_list)

Done:20/1
Done:20/2
Done:20/3
Done:20/4
Done:20/5
Done:20/6
Done:20/7
Done:20/8
Done:20/9
Done:20/10
Done:20/11
Done:20/12
Done:20/13
Done:20/14
Done:20/15
Done:20/16
Done:20/17
Done:20/18
Done:20/19
Done:20/20
All done


In [24]:
saving_test_path = 'C:\\Users\\homol\\Documents\\DL\\DL_BHW\\data\\GT_TEST\\'
get_2d_images_gt(saving_test_path,test_files_list)

Done:10/1
Done:10/2
Done:10/3
Done:10/4
Done:10/5
Done:10/6
Done:10/7
Done:10/8
Done:10/9
Done:10/10
All done
